In [24]:
import requests 
import json
import os

In [25]:
#Api urls 
pdf_to_html = 'https://auth.anuvaad.org/api/v0/pdf-to-html'
htm_to_json = 'https://auth.anuvaad.org/api/v0/html-to-json'

In [36]:
#directories and files

logs = '/home/dhiraj/note_book/v4/logs.txt'
json_dir ='/home/dhiraj/Documents/Anuwad/page_segmentation/v4/sample_pdfs/pdf_to_html_to_json/'
file_id = '0018022427'


In [27]:

def post_request(url ,param):
    
    param = json.dumps(param)
    headers = {"Content-Type": "application/json"}
    print(param)
    r = requests.post(url, data = param , headers=headers )
    return r.json()

In [ ]:

def img_to_df(image):
    text = pytesseract.image_to_data(image, lang='eng',output_type=Output.DATAFRAME)
    return text


###############################################
def show_text_box(input_image,lines_data):
    output_image = np.ones((input_image.shape[0],input_image.shape[1]))
    for index,line in lines_data.iterrows():
            cv2.rectangle(output_image, (line['left'], line['top']), (line['right'], line['bottom']), (0), 2)

    plt.imshow(output_image)
    return output_image
#############################################

def bloat_text(image,threshold):
    #converitng image to binary
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image>125
    image = image.astype(np.uint8)
    #Bloating
    dist_transform   = cv2.distanceTransform(image,cv2.DIST_L1,5)
    ret, sure_fg     = cv2.threshold(dist_transform,threshold,255,0)
    plt.imshow(sure_fg)
    return sure_fg.astype(np.uint8)


def sort_words(group,sorted_group=[],line_spacing=[],line=0):

    semi_height        = group.iloc[0]['height'].mean() / 2.0
    check_y            = group.iloc[0]['top']
    same_line          = group[ abs(group['top'] - check_y) < semi_height]
    next_lines         = group[ abs(group['top'] - check_y) >= semi_height]
    sort_lines         = same_line.sort_values(by=['left'])
    sort_lines['line'] = line
    line_spacing.append(same_line['ymid'].mean())
    line +=1
    for index, row in sort_lines.iterrows():
        sorted_group.append(row)

    if len(next_lines) > 0 :
        sort_words (next_lines, sorted_group,line_spacing,line)

    return sorted_group , line_spacing,line



def sort_contours(contours_df,spacing,sorted_contours=[],line=1):

    check_y          = contours_df.iloc[0]['top']
    same_line        = contours_df[ abs(contours_df['top'] - check_y) < spacing]
    next_lines       = contours_df[ abs(contours_df['top'] - check_y) >= spacing]
    sort_lines       = same_line.sort_values(by=['left'])
    for index, row in sort_lines.iterrows():
        #print(line,row['top'],row['height'], check_y,spacing)
        sorted_contours.append(row)
    line +=1 
    if len(next_lines) > 0 :
        sort_contours(next_lines,spacing, sorted_contours,line)

    return sorted_contours

def median_spacing(line_spacing):
    spacing = []
    for i in range(len(line_spacing) - 1):
        spacing.append(line_spacing[i+1] -line_spacing[i])
    spacing_median = np.median(spacing)
    
    return spacing_median
    
    
def find_and_sort_contours(bloated_image,line_spacing_median):
    contours = cv2.findContours (bloated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours [0] if len (contours) == 2 else contours [1]
    image_area = bloated_image.shape[0] * bloated_image.shape[1]
    contours_list =[]
    for c in contours:
        x, y, w, h = cv2.boundingRect (c)
        if (w*h) < (image_area * 0.95):
            contours_list.append([x,y,w,h])
    contours_df = pd.DataFrame(contours_list,columns=['left','top','width','height'])
    contours_df = contours_df.sort_values(by=['top'])
    sorted_contours = sort_contours(contours_df,line_spacing_median*0.5,[])
    sorted_contours =pd.DataFrame(sorted_contours).reset_index()

    return sorted_contours

def extract_region(sorted_df,row):
    sub_text_df = sorted_df[(sorted_df['left'] >= row['left']) & (sorted_df['right'] <= (row['left'] +row['width'])) & (sorted_df['top']>=row['top']) & (sorted_df['bottom']<=(row['top']+row['height'])) ]
    #sub_text_df = sorted_df[(sorted_df['top']>=row['top']) & (sorted_df['bottom']<=(row['top']+row['height']))]
    return sub_text_df
    
    
def line_start_and_end_stats(sorted_df):
    lines = sorted_df['line'].unique()
    lines_df = pd.DataFrame(lines,columns=['line'])
    lines_df['start'] = 0
    lines_df['end']   = 0
    for line in lines:
        sub_sorted_df             =  sorted_df[sorted_df['line']==line]
        lines_df['start'][line]   = sub_sorted_df['left'].min()
        sub_sorted_df['word_end'] = sub_sorted_df['left'] + sub_sorted_df['width']
        lines_df['end'][line]     = sub_sorted_df['word_end'].max()
        
    return lines_df



def extraction_helper(input_image):
    text_df = img_to_df(input_image)
    text_df = text_df[text_df['conf'] > tesseract_conf]
    if len(text_df) > 0 :
        text_df['bottom'] =text_df['top'] + text_df['height']
        text_df['right']  = text_df['left'] + text_df['width']
        text_df['ymid']   = text_df['top'] + text_df['height'] * 0.5
        text_df           = text_df.sort_values(by=['top'])
        text_df['text']   = text_df['text'].astype(str)
        text_df['line']   = None
        median_height     = text_df['height'].median()
        text_df           = text_df[text_df['height'] > (median_height / 4.0)]


        sorted_df =[]
        line_spacing =[]
        line  = 0

        sorted_df,line_spacing,line = sort_words(text_df,sorted_df,line_spacing,line)
        ####################################
        box_image = show_text_box(input_image,text_df)
        
        ####################################
        sorted_df            = pd.DataFrame(sorted_df).reset_index()
        line_df              = line_start_and_end_stats(sorted_df)
        line_spacing_median  = median_spacing(line_spacing)
        bloated_image        = bloat_text(box_image *255,line_spacing_median * 0.5)
        sorted_contours      = find_and_sort_contours(bloated_image,line_spacing_median)

        return sorted_df,sorted_contours,line_df,line_spacing_median
    else :
        return None, None,None,None



def get_left_margin(line_df,sorted_df,margin_support):
    line_start_distribution = np.array(sorted(line_df['start']))
    threshold = sorted_df['height'].median()
    #check support

    for starting_point in line_start_distribution:
        delta  = abs(line_start_distribution - starting_point)
        margin_delta = delta < threshold
        if margin_delta.sum() > margin_support:
            break

    return starting_point





def get_right_margin(line_df,sorted_df,margin_support):
    line_end_distribution = np.array(sorted(line_df['end'] ,reverse=True))
    threshold = sorted_df['height'].median()
    #check support

    for end_point in line_end_distribution:
        delta  = abs(line_end_distribution - end_point)
        margin_delta = delta < threshold
        if margin_delta.sum() > margin_support:
            break

    return end_point

def line_parser(sorted_contours,line_df,sorted_df,margin_support, page_number,pdf_index):
    lines_data=[]
    #page_number = 1
    #pdf_index =   0
    for index,row in sorted_contours.iterrows():
        extracted_region = extract_region(sorted_df,row)
        blob_start       = row['left']
        if len(extracted_region) > 0 :
            #print(extracted_region)
            lines_in_blob  =  extracted_region['line'].unique()
            lines_count    = len(lines_in_blob)
            first_line = lines_in_blob[0]
            last_line  = lines_in_blob[-1]
            for line_id in lines_in_blob:
                line= {}
                same_line               =  extracted_region[extracted_region['line'] == line_id]
                #print(same_line)
                line['text']            =  ' '.join(same_line['text'].values)
                line['top']             =  int(same_line['top'].min())
                line['left']            =  int(same_line['left'].min())
                line['height']          =  int(same_line['height'].max())
                line['right']           =  int(same_line['right'].max())
                line['bottom']          =  int(same_line['bottom'].max())

                line['block_num']       =  int(same_line['block_num'].iloc[0])
                line['blob_id']         =  index
                line['pdf_index']       =  pdf_index
                line['page_no']         =  page_number
                line['avrage_conf']     =  float(same_line['conf'].mean())
                line['page_line_index'] = line
                line['visual_break']    = break_condition(line_df,sorted_df,margin_support,line_id,last_line,blob_start, page_number,lines_count)

                pdf_index += 1
                lines_data.append(line)

    return lines_data,pdf_index






#visual_break
def break_condition(line_df,sorted_df,margin_support,line_id,last_line,blob_start, page_number,lines_count):
    left_margin = get_left_margin(line_df,sorted_df,margin_support)
    right_margin =  get_right_margin(line_df,sorted_df,margin_support)
    median_word_height = sorted_df['height'].median()
    line_start  = line_df['start'][line_id] 
    line_ending = line_df['end'][line_id]
    start_delta = abs(line_start - left_margin)
    end_delta = abs(right_margin - line_ending)
    
    if line_id == last_line :
        return 1
    else :
        #First pages uses centre alignment fo headings and titles 
        if page_number ==1:
            if start_delta > 3* median_word_height:
                 if end_delta > 2*median_word_height:
                    return 1
            else:
                if lines_count > 3:
                    if end_delta > 2*median_word_height:
                        return 1                                    
        else :
            # Supreme court uses justified text alignment 
            if start_delta <  2* median_word_height:
                if end_delta > 2*median_word_height:
                    return 1
            else :
                if abs(line_start - line_df['start'][line_id+1] ) > 2*median_word_height:
                    if abs(line_ending - line_df['end'][line_id +1]) >  2*median_word_height:
                        return 1
                    


    return 0



def show_breaks(input_image,lines_data):
    output_image = input_image.copy()
    for line in lines_data:
        if line['visual_break']:
            cv2.rectangle(output_image, (line['left'], line['top']), (line['right'], line['bottom']), (255,0,0), 2)

        else :
             cv2.rectangle(output_image, (line['left'], line['top']), (line['right'], line['bottom']), (0,255,0), 2)
                
    
    return output_image


In [28]:
#file_id = '0022610323'
pdf_to_html_input = {"input": {"files":[{ "locale": "en","path": file_id,"type": "pdf"}]},"jobID": "Pdf-12354687954","state": "INITIATED","status": "STARTED","stepOrder": 0,"workflowCode": "DP_WFLOW_P","tool":"PDF2HTML"}

In [29]:
phml = post_request(pdf_to_html,pdf_to_html_input)
path_to_html = phml['output']['files'][0]['outputHtmlFilePath']

{"input": {"files": [{"locale": "en", "path": "0018022427", "type": "pdf"}]}, "jobID": "Pdf-12354687954", "state": "INITIATED", "status": "STARTED", "stepOrder": 0, "workflowCode": "DP_WFLOW_P", "tool": "PDF2HTML"}


In [30]:
htm_to_json_input = {
    "status": "SUCCESS",
    "state": "HTML-TO-JSON-PROCESSED",
    "jobID": "HTML-12354687954",
    "taskID": "Html2Json-15928305372734475",
    "workflowCode": "DP_WFLOW_P",
    "input": {
        "files": [
            {
                "path": path_to_html,
                "locale": "en",
                "type": "folder"
            }
        ]
    },
    "tool": "HTML2JSON",
    "stepOrder": 0
}

In [31]:
pdf_json = post_request(htm_to_json,htm_to_json_input)

{"status": "SUCCESS", "state": "HTML-TO-JSON-PROCESSED", "jobID": "HTML-12354687954", "taskID": "Html2Json-15928305372734475", "workflowCode": "DP_WFLOW_P", "input": {"files": [{"path": "1594036339637121/html_files", "locale": "en", "type": "folder"}]}, "tool": "HTML2JSON", "stepOrder": 0}


In [34]:
print(file_id)
json_path = pdf_json['output']['files'][0]['outputHtmlFilePath']

In [ ]:
json_path = os.path.join(json_dir,json_path)

In [ ]:
with open(json_path,'rb') as f:
  data = json.load(f)

In [ ]:
data[0]['html_nodes'][3]

In [ ]:
file_id

In [ ]:
response_json = os.path.join(json_dir,file_id + '.json')

In [ ]:
with open(response_json,'rb') as f:
 response_data= json.load(f)

In [ ]:
response_data

In [ ]:
data

In [20]:
#plot 
def load_json(path,file_id):
    with open(path + file_id +'.json','rb') as f:
        data = json.load(f)
    return data    


    
    

def show_text_box(input_image,lines_data):
    output_image = np.ones((input_image.shape[0],input_image.shape[1]))*255
    for index,line in lines_data.iterrows():
            cv2.rectangle(output_image, (line['left'], line['top']), (line['right'], line['bottom']), (0), 2)

    plt.imshow(output_image)
    return output_image.astype(np.uint8)

In [39]:
json_ouput = load_json(json_dir , file_id)

In [40]:
json_ouput

[{'html_nodes': [{'page_no': '1',
    'x': '454',
    'y': '54',
    'class': 'ft00',
    'class_style': {'font-size': '16px',
     'font-family': 'BAAAAA+DejaVuSans',
     'color': '#000000'},
    'style': 'position:absolute;top:54px;left:454px;white-space:nowrap',
    'text': '1',
    'is_bold': False},
   {'page_no': '1',
    'x': '668',
    'y': '109',
    'class': 'ft01',
    'class_style': {'font-size': '21px',
     'font-family': 'CAAAAA+LiberationSerif',
     'color': '#000000'},
    'style': 'position:absolute;top:109px;left:668px;white-space:nowrap',
    'text': 'REPORTABLE',
    'is_bold': True},
   {'page_no': '1',
    'x': '288',
    'y': '157',
    'class': 'ft02',
    'class_style': {'font-size': '21px',
     'font-family': 'DAAAAA+LiberationSerif',
     'color': '#000000'},
    'style': 'position:absolute;top:157px;left:288px;white-space:nowrap',
    'text': 'IN\xa0THE\xa0SUPREME\xa0COURT\xa0OF INDIA',
    'is_bold': False},
   {'page_no': '1',
    'x': '274',
    'y': 